# Traditional DoE vs. Bayesian Optimization

The [first](https://youtu.be/Evua529dAgc) and
[second](https://youtu.be/41fQs4JxRQA) videos from Taylor Sparks' [Optimization Tutorial
YouTube playlist](https://youtube.com/playlist?list=PLL0SWcFqypClTIMQDOs_Jug70qaVPOzEc)
are based on the results from this notebook. I suggest watching these two videos prior
to working through this notebook for better context.

This notebook first describes uninformed
sampling methods including random, grid, and quasi-random (e.g., Latin hypercube, Sobol
sequences) and shows how quasi-random achieves a much more even distribution of points.
Finally, we compare the efficiency of these traditional methods for design of
experiments (DoE) against Bayesian optimization. We show that, on average, Bayesian
optimization is much more efficient than traditional DoE.

In [2]:
! pip install plotly

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 19.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [4]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import qmc
from os import path

In [5]:
bounds = {"x1": [0, 1], "x2": [0, 1]}
num_samples = 10

## Uninformed Sampling Methods

These are sampling methods that do not incorporate information about the objective function
to be optimized. We will cover grid search, random search, and two quasi-random methods:
Latin hypercube and Sobol sequences.

### Grid

Grid sampling is a structured approach to sampling from a search space. It involves
creating a grid of points over the space, and then selecting points from the grid. When
grids are used for certain types of problems (e.g., finite element methods), the
algorithms are often more straightforward than for non-lattice point sets. This can be
an effective way to ensure that the entire search space is explored, but it can also
lead to a lot of unnecessary points being generated, especially for high-dimensional
search spaces, due to large, systematic "pockets" in the search space.


In [6]:
from sklearn.model_selection import ParameterGrid

def get_grid_samples(bounds, num_samples = 10, seed=None):
    # seed is unused, for compatibility only
    param_grid = {}
    num_pts_per_dim = max(1, np.floor(num_samples ** (1 / len(bounds))).astype(int))
    for name, bnd in bounds.items():
        param_grid[name] = np.linspace(bnd[0], bnd[1], num=num_pts_per_dim)
    print(num_pts_per_dim)
    return pd.DataFrame(list(ParameterGrid(param_grid)))

grid_samples = get_grid_samples(bounds, num_samples=num_samples)
grid_samples

3


,x1,x2
0,0.0,0.0
1,0.0,0.5
2,0.0,1.0
3,0.5,0.0
4,0.5,0.5
5,0.5,1.0
6,1.0,0.0
7,1.0,0.5
8,1.0,1.0


In [12]:
grid_fig = px.scatter(grid_samples, x="x1", y="x2", width=400, height=400)
grid_fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'x1=%{x}<br>x2=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([0. , 0. , 0. , 0.5, 0.5, 0.5, 1. , 1. , 1. ]),
              'xaxis': 'x',
              'y': array([0. , 0.5, 1. , 0. , 0.5, 1. , 0. , 0.5, 1. ]),
              'yaxis': 'y'}],
    'layout': {'height': 400,
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'width': 400,
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'x1'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'x2'}}}
})

### Random

Random sampling is a simple and straightforward method for generating samples from a
search space. It involves randomly selecting points from the space, without any regard
for their distribution. This can be an effective method for exploring search spaces, and
it is often more effective than grid search. While it doesn't have the large, systematic
pockets characteristic of grid search, it has large, occasional gaps due to the random
nature of the search.

In [8]:
from numpy.random import default_rng

def get_random_samples(bounds, num_samples=9, seed=None):
    rng = default_rng(seed)
    samples = {}
    for parameter, bound in bounds.items():
        samples[parameter] = rng.uniform(bound[0], bound[1], num_samples)
    return pd.DataFrame(samples)

random_samples = get_random_samples(bounds, seed=0)
random_samples

,x1,x2
0,0.636962,0.935072
1,0.269787,0.815854
2,0.040974,0.002739
3,0.016528,0.857404
4,0.813270,0.033586
5,0.912756,0.729655
6,0.606636,0.175656
7,0.729497,0.863179
8,0.543625,0.541461


In [6]:
random_fig = px.scatter(random_samples, x="x1", y="x2", width=400, height=400)
random_fig

## Quasi-Random

Quasi-random sampling methods, also known as low-discrepancy or deterministic sampling methods, are a family of sampling techniques that are designed to
produce samples that are more evenly distributed than random samples. Unlike random
sampling, which selects points randomly and independently, quasi-random sampling methods
aim to achieve a more uniform coverage of the parameter space by reducing the
**discrepancy** between the generated points and the desired distribution.

We'll discuss two common quasi-random sampling methods: Latin hypercube and
Sobol sequences.

### Latin Hypercube

Latin hypercube sampling (LHS) is a variation of grid sampling that aims to improve the
uniformity of the samples. It does this by ensuring that each dimension of the search
space is represented equally in the sample set. It involves dividing the parameter space
into equally spaced intervals and randomly selecting one point within each interval. LHS
ensures a more even coverage of the parameter space compared to random or grid sampling
methods (i.e., lower discrepancy).

In [7]:
def get_latin_hypercube_samples(bounds, num_samples=10, seed=None):
    sampler = qmc.LatinHypercube(d=len(bounds), optimization="random-cd", seed=seed)
    samples = sampler.random(num_samples)
    l_bounds = [bound[0] for bound in bounds.values()]
    u_bounds = [bound[1] for bound in bounds.values()]
    samples = qmc.scale(samples, l_bounds, u_bounds)
    return pd.DataFrame(samples, columns=list(bounds.keys()))

latin_hypercube_samples = get_latin_hypercube_samples(bounds, seed=0)
latin_hypercube_samples

,x1,x2
0,0.245638,0.173021
1,0.436304,0.298347
2,0.927034,0.708724
3,0.114260,0.827050
4,0.718673,0.006493
5,0.013682,0.499726
6,0.595903,0.996641
7,0.639336,0.582434
8,0.318415,0.645854
9,0.870029,0.357731


In [8]:
latin_hypercube_fig = px.scatter(
    latin_hypercube_samples, x="x1", y="x2", width=400, height=400
)
latin_hypercube_fig


### Comparison between sampling methods

The quasi-random methods tend to have better space-filling properties than random or
grid search. Note the large systematic gaps in grid, the large occasional gaps in
random, and the more even distribution of points in LHS and Sobol.

In [11]:
sampling_fns = dict(
    grid=get_grid_samples,
    random=get_random_samples,
    latin_hypercube=get_latin_hypercube_samples,
    sobol=get_sobol_samples,
)

sample_nums = [5, 10, 50, 100]
sample_nums.reverse()
        
sample_dfs = []
for name, sampling_fn in sampling_fns.items():
    for num_samples in sample_nums:
        sample_df = sampling_fn(bounds, num_samples)
        sample_df["name"] = name
        sample_df["num_samples"] = num_samples
        sample_dfs.append(sample_df)

compare_df = pd.concat(sample_dfs, axis=0)

10
7
3
2


C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.



In [12]:
compare_df

,x1,x2,name,num_samples
0,0.000000,0.000000,grid,100
1,0.000000,0.111111,grid,100
2,0.000000,0.222222,grid,100
3,0.000000,0.333333,grid,100
4,0.000000,0.444444,grid,100
...,...,...,...,...
0,0.924471,0.379841,sobol,5
1,0.321745,0.816658,sobol,5
2,0.119724,0.066057,sobol,5
3,0.626250,0.628000,sobol,5


In [13]:
fig = px.scatter(
    compare_df,
    x="x1",
    y="x2",
    facet_row="num_samples",
    facet_col="name",
    width=800,
    height=800,
)
plot_and_save(
    "traditional-doe-compare",
    fig,
    show=True,
    mpl_kwargs=dict(width_inches=7.5, height_inches=8.0),
)


### Worsening performance in higher dimensions

As we observe the discrepancy associated with sampling methods in higher dimensions, we
notice that the gap between the quasi-random methods and the random and grid methods
widens. In other words, quasi-random methods increasingly outperform random and grid as
the dimensionality increases.

In [14]:
one = get_grid_samples(dict(x1=bounds["x1"]), num_samples=3**1)
two = get_grid_samples(dict(x1=bounds["x1"], x2=bounds["x2"]), num_samples=3**2)
three = get_grid_samples(
    dict(x1=bounds["x1"], x2=bounds["x2"], x3=[0.0, 1.0]), num_samples=3**3
)


3
3
3


In [15]:
# https://community.plotly.com/t/plotting-a-simple-1d-number-line/39169/4
import plotly.graph_objects as go
fig = go.Figure()
x = one["x1"]
fig.add_trace(go.Scatter(
    x=x, y=[0] * len(x), mode='markers', marker_size=20,
))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False, 
                 zeroline=True, zerolinecolor='black', zerolinewidth=3,
                 showticklabels=False)
fig.update_layout(height=200, plot_bgcolor='white')
fig.show()

# px.scatter(one, x="x1", y=[0]*3)

In [16]:
px.scatter(two, x="x1", y="x2", width=400, height=400)

In [17]:
# https://community.plotly.com/t/rotating-3d-plots-with-plotly/34776/2
# https://community.plotly.com/t/how-to-export-animation-and-save-it-in-a-video-format-like-mp4-mpeg-or/64621/2
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio

x, y, z = three["x1"], three["x2"], three["x3"]

fig = go.Figure(go.Scatter3d(x=x, y=y, z=z, mode="markers"))

x_eye = -1.25
y_eye = 2
z_eye = 1.0

fig.update_layout(
    title="Animation Test",
    width=600,
    height=600,
    scene_camera_eye=dict(x=x_eye, y=y_eye, z=z_eye),
    updatemenus=[
        dict(
            type="buttons",
            showactive=False,
            y=1,
            x=0.8,
            xanchor="left",
            yanchor="bottom",
            pad=dict(t=45, r=10),
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[
                        None,
                        dict(
                            frame=dict(duration=5, redraw=True),
                            transition=dict(duration=0),
                            fromcurrent=True,
                            mode="immediate",
                        ),
                    ],
                )
            ],
        )
    ],
)


def rotate_z(x, y, z, theta):
    w = x + 1j * y
    return np.real(np.exp(1j * theta) * w), np.imag(np.exp(1j * theta) * w), z


frames = []
pil_frames = []
for t in np.arange(0, 3.14, 0.025):
    xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
    frames.append(go.Frame(layout=dict(scene_camera_eye=dict(x=xe, y=ye, z=ze))))
fig.frames = frames

fig.show()

In [18]:
[qmc.discrepancy(df.values) for df in [one, two, three]]

[0.0277777777777779, 0.060956790123456894, 0.10033007544581585]

In [19]:
discrepancies = []
for name, sampling_fn in sampling_fns.items():
    for num_samples in sample_nums:
        sample_df = compare_df.query("name == @name and num_samples == @num_samples")
        discrepancies.append(
            dict(
                name=name,
                num_samples=num_samples,
                discrepancy=qmc.discrepancy(sample_df[["x1", "x2"]].values),
            )
        )
        
discrepancy_df = pd.DataFrame(discrepancies)
discrepancy_df

,name,num_samples,discrepancy
0,grid,100,0.004093
1,grid,50,0.008708
2,grid,10,0.060957
3,grid,5,0.204861
4,random,100,0.002874
5,random,50,0.011180
6,random,10,0.033466
7,random,5,0.098165
8,latin_hypercube,100,0.000058
9,latin_hypercube,50,0.000224


In [20]:
fig = px.scatter(
    compare_df,
    x="x1",
    y="x2",
    facet_row="num_samples",
    facet_col="name",
    width=800,
    height=800,
)

for col, (name, sampling_fn) in enumerate(sampling_fns.items()):
    col = col+1
    for row, num_samples in enumerate(sample_nums):
        row = 4 - row
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=-0.1,
            text=f' Discrepancy = {discrepancy_df.query("name == @name and num_samples == @num_samples").iloc[0]["discrepancy"]:.3g} ',
            # text = f"row={row}, col={col}",
            showarrow=False,
            bgcolor="white",
            row=row,
            col=col,
        )

fig_path = "traditional-doe-compare-discrepancy"
fig.update_layout(
    margin=dict(r=40, t=30, b=30),
)
fig.write_html(fig_path + ".html")
fig.write_image(fig_path + ".png")
fig.show()


In [21]:
dim_discrepancies = []
# sample_dfs = []
dim_nums = [2, 3, 10, 20]
num_samples = 100
for name, sampling_fn in sampling_fns.items():
    for num_dims in dim_nums:
        bounds = {f"x{i+1}": [0, 1] for i in range(num_dims)}
        sample_df = sampling_fn(bounds, num_samples, seed=0)
        discrepancy = qmc.discrepancy(sample_df.values)
        dim_discrepancies.append(dict(name=name, num_samples=sample_df.shape[0], discrepancy=discrepancy, num_dims=num_dims))
        # sample_dfs.append(sample_df)

dim_discrepancy_df = pd.DataFrame(dim_discrepancies)
pd.pivot_table(
    dim_discrepancy_df.drop("num_samples", axis=1),
    index=["num_dims", "discrepancy", "name"],
)


10
4
1
1


C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.



Empty DataFrame
Columns: []
Index: [(2, 6.409265931739405e-05, latin_hypercube), (2, 0.00013419647223589237, sobol), (2, 0.004092935528052344, grid), (2, 0.006477791186335402, random), (3, 0.0002103949356229151, latin_hypercube), (3, 0.0003235892122264161, sobol), (3, 0.010809351752242069, random), (3, 0.05335594564475454, grid), (10, 0.018653897714654644, latin_hypercube), (10, 0.022224334095691134, sobol), (10, 0.06842806743129515, random), (10, 53.396888619801956, grid), (20, 0.45423472936956166, latin_hypercube), (20, 0.5717774649791076, sobol), (20, 0.8405888642021928, random), (20, 3309.1238072763103, grid)]

In [22]:
dim_discrepancies = []
# sample_dfs = []
dim_nums = [2, 3, 10, 20]
num_samples = 10
for name, sampling_fn in sampling_fns.items():
    for num_dims in dim_nums:
        bounds = {f"x{i+1}": [0, 1] for i in range(num_dims)}
        sample_df = sampling_fn(bounds, num_samples, seed=0)
        discrepancy = qmc.discrepancy(sample_df.values)
        dim_discrepancies.append(dict(name=name, num_samples=sample_df.shape[0], discrepancy=discrepancy, num_dims=num_dims))
        # sample_dfs.append(sample_df)

dim_discrepancy_df = pd.DataFrame(dim_discrepancies)
pd.pivot_table(
    dim_discrepancy_df.drop("num_samples", axis=1),
    index=["num_dims", "discrepancy", "name"],
)


3
2
1
1


C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.



Empty DataFrame
Columns: []
Index: [(2, 0.005236766705729812, latin_hypercube), (2, 0.0076472324046004125, sobol), (2, 0.019627671148842563, random), (2, 0.060956790123456894, grid), (3, 0.011434040866014517, latin_hypercube), (3, 0.01451538370993033, sobol), (3, 0.04586154399146114, random), (3, 0.37688078703703676, grid), (10, 0.30414326400414193, latin_hypercube), (10, 0.43743273730096544, sobol), (10, 0.9976404884270602, random), (10, 53.396888619801956, grid), (20, 6.131461128328029, latin_hypercube), (20, 7.369505984936929, sobol), (20, 11.771547486969345, random), (20, 3309.1238072763103, grid)]

In [23]:
dim_discrepancies = []
# sample_dfs = []
dim_nums = [2, 3, 10, 20]
num_samples = 1000
for name, sampling_fn in sampling_fns.items():
    for num_dims in dim_nums:
        bounds = {f"x{i+1}": [0, 1] for i in range(num_dims)}
        sample_df = sampling_fn(bounds, num_samples, seed=0)
        discrepancy = qmc.discrepancy(sample_df.values)
        dim_discrepancies.append(dict(name=name, num_samples=sample_df.shape[0], discrepancy=discrepancy, num_dims=num_dims))
        # sample_dfs.append(sample_df)

dim_discrepancy_df = pd.DataFrame(dim_discrepancies)
pd.pivot_table(
    dim_discrepancy_df.drop("num_samples", axis=1),
    index=["num_dims", "discrepancy", "name"],
)


31
9
1
1


C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_29488\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.



Empty DataFrame
Columns: []
Index: [(2, 1.3837207570865928e-06, latin_hypercube), (2, 1.6838481025072127e-06, sobol), (2, 0.0003923703424881797, grid), (2, 0.001073323450584196, random), (3, 4.8907254588392846e-06, sobol), (3, 8.754559247092786e-06, latin_hypercube), (3, 0.0013353502970596676, random), (3, 0.008351056258295353, grid), (10, 0.0008654031158545905, sobol), (10, 0.001360454305847636, latin_hypercube), (10, 0.007363398310313407, random), (10, 53.396888619801956, grid), (20, 0.043370026241586146, sobol), (20, 0.04457565812541375, latin_hypercube), (20, 0.07909763543233517, random), (20, 3309.1238072763103, grid)]